# Spark (cluster computing)

In [109]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('flights analysis').getOrCreate()


# Read thr DF

In [128]:
flights = spark.read.csv('flights.csv',
                        sep=',',
                        header=True,
                        inferSchema=True,
                       nullValue='NA')
flights.show()

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| NULL|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| NULL|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|
|  1| 16|  6|     UA|  1477|ORD|1440|   8.0|     232|   -7|
|  1| 22|  5|     UA|   620|SJC|1829|  7.98|     250|  -13|
| 11|  8|  1|     OO|  5590|SFO| 158|  7.77|      60|   88|
|  4| 26|  1|     AA|  1144|SFO|1464| 13.25|     210|  -10|
|  4| 25|  0|     AA|   321|ORD| 978| 13.75|     160|   31|
|  8| 30|  2|     UA|   646|ORD| 719| 13.28|     151|   16|
|  3| 16|  3|     UA|   107|ORD|1745|   

# Remove Col

In [129]:
flights = flights.drop('flight')

# check nulls and remove nulls

In [130]:
flights.filter('delay is null').count()

2978

In [131]:
flights = flights.filter('delay is not null')
flights.filter('delay is null').count()

0

# Columns manipulation

In [132]:
from pyspark.sql.functions import round

flights = flights.withColumn('km',round(flights.mile*1.60934,0)).drop('mile')       

flights = flights.withColumn('label',(flights.delay>=15).cast('integer')).drop('delay')

flights.show()

+---+---+---+-------+---+------+--------+------+-----+
|mon|dom|dow|carrier|org|depart|duration|    km|label|
+---+---+---+-------+---+------+--------+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|      82| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|      82| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|     195|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|     102| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|     135|1180.0|    1|
|  1| 16|  6|     UA|ORD|   8.0|     232|2317.0|    0|
|  1| 22|  5|     UA|SJC|  7.98|     250|2943.0|    0|
| 11|  8|  1|     OO|SFO|  7.77|      60| 254.0|    1|
|  4| 26|  1|     AA|SFO| 13.25|     210|2356.0|    0|
|  4| 25|  0|     AA|ORD| 13.75|     160|1574.0|    1|
|  8| 30|  2|     UA|ORD| 13.28|     151|1157.0|    1|
|  3| 16|  3|     UA|ORD|   9.0|     264|2808.0|    0|
|  0|  3|  4|     AA|LGA| 17.08|     190|1765.0|    1|
|  5|  9|  1|     UA|SFO|  12.7|     158|1556.0|    1|
|  3| 10|  4|     B6|ORD| 17.58|     265|2792.0|    1|
| 11| 15| 

# Deal with categorical columns


In [133]:
flights.dtypes

[('mon', 'int'),
 ('dom', 'int'),
 ('dow', 'int'),
 ('carrier', 'string'),
 ('org', 'string'),
 ('depart', 'double'),
 ('duration', 'int'),
 ('km', 'double'),
 ('label', 'int')]

In [134]:
from pyspark.ml.feature import StringIndexer 

flights = StringIndexer(inputCol='carrier',outputCol='carrier_idx').fit(flights).transform(flights)#.drop('carrier')
flights.select('carrier','carrier_idx').show()

+-------+-----------+
|carrier|carrier_idx|
+-------+-----------+
|     UA|        0.0|
|     UA|        0.0|
|     AA|        1.0|
|     UA|        0.0|
|     AA|        1.0|
|     UA|        0.0|
|     UA|        0.0|
|     OO|        2.0|
|     AA|        1.0|
|     AA|        1.0|
|     UA|        0.0|
|     UA|        0.0|
|     AA|        1.0|
|     UA|        0.0|
|     B6|        4.0|
|     AA|        1.0|
|     UA|        0.0|
|     B6|        4.0|
|     OO|        2.0|
|     OO|        2.0|
+-------+-----------+
only showing top 20 rows



In [135]:
flights = StringIndexer(inputCol='org',outputCol='org_idx').fit(flights).transform(flights)
flights.select('org','org_idx').show()

+---+-------+
|org|org_idx|
+---+-------+
|ORD|    0.0|
|SFO|    1.0|
|ORD|    0.0|
|SFO|    1.0|
|ORD|    0.0|
|ORD|    0.0|
|SJC|    5.0|
|SFO|    1.0|
|SFO|    1.0|
|ORD|    0.0|
|ORD|    0.0|
|ORD|    0.0|
|LGA|    3.0|
|SFO|    1.0|
|ORD|    0.0|
|ORD|    0.0|
|SJC|    5.0|
|JFK|    2.0|
|ORD|    0.0|
|SFO|    1.0|
+---+-------+
only showing top 20 rows



In [136]:
flights = flights.drop('carrier','org')

# Vector assembler

In [137]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
            inputCols=['mon','dom','dow','depart','duration','km','carrier_idx','org_idx'],
            outputCol = 'features')
flights = assembler.transform(flights)

In [138]:
flights.show()

+---+---+---+------+--------+------+-----+-----------+-------+--------------------+
|mon|dom|dow|depart|duration|    km|label|carrier_idx|org_idx|            features|
+---+---+---+------+--------+------+-----+-----------+-------+--------------------+
|  0| 22|  2| 16.33|      82| 509.0|    1|        0.0|    0.0|[0.0,22.0,2.0,16....|
|  2| 20|  4|  6.17|      82| 542.0|    0|        0.0|    1.0|[2.0,20.0,4.0,6.1...|
|  9| 13|  1| 10.33|     195|1989.0|    0|        1.0|    0.0|[9.0,13.0,1.0,10....|
|  5|  2|  1|  7.98|     102| 885.0|    0|        0.0|    1.0|[5.0,2.0,1.0,7.98...|
|  7|  2|  6| 10.83|     135|1180.0|    1|        1.0|    0.0|[7.0,2.0,6.0,10.8...|
|  1| 16|  6|   8.0|     232|2317.0|    0|        0.0|    0.0|[1.0,16.0,6.0,8.0...|
|  1| 22|  5|  7.98|     250|2943.0|    0|        0.0|    5.0|[1.0,22.0,5.0,7.9...|
| 11|  8|  1|  7.77|      60| 254.0|    1|        2.0|    1.0|[11.0,8.0,1.0,7.7...|
|  4| 26|  1| 13.25|     210|2356.0|    0|        1.0|    1.0|[4.0,26.0,1.0,

# splitting 

In [143]:
flights_train ,flights_test= flights.randomSplit([0.8,0.2],seed=43)

# model

In [144]:
from pyspark.ml.classification import DecisionTreeClassifier

tree = DecisionTreeClassifier(labelCol='label', featuresCol='features')

tree_model = tree.fit(flights_train)

pred = tree_model.transform(flights_test)

In [145]:
pred.show(5)

+---+---+---+------+--------+------+-----+-----------+-------+--------------------+---------------+--------------------+----------+
|mon|dom|dow|depart|duration|    km|label|carrier_idx|org_idx|            features|  rawPrediction|         probability|prediction|
+---+---+---+------+--------+------+-----+-----------+-------+--------------------+---------------+--------------------+----------+
|  0|  1|  2|   6.0|      82| 600.0|    1|        2.0|    4.0|[0.0,1.0,2.0,6.0,...|  [927.0,132.0]|[0.87535410764872...|       0.0|
|  0|  1|  2|  6.25|     120| 999.0|    1|        6.0|    5.0|[0.0,1.0,2.0,6.25...|  [927.0,132.0]|[0.87535410764872...|       0.0|
|  0|  1|  2|   6.5|     240|2235.0|    1|        1.0|    3.0|[0.0,1.0,2.0,6.5,...|[1482.0,1280.0]|[0.53656770456191...|       0.0|
|  0|  1|  2|  7.25|     122|1091.0|    1|        6.0|    0.0|[0.0,1.0,2.0,7.25...|[1482.0,1280.0]|[0.53656770456191...|       0.0|
|  0|  1|  2|  8.08|     120|1180.0|    0|        1.0|    0.0|[0.0,1.0,2.0,8

In [146]:
pred.select('label','prediction','probability').show()

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    1|       0.0|[0.87535410764872...|
|    1|       0.0|[0.87535410764872...|
|    1|       0.0|[0.53656770456191...|
|    1|       0.0|[0.53656770456191...|
|    0|       0.0|[0.53656770456191...|
|    1|       0.0|[0.53656770456191...|
|    0|       0.0|[0.53656770456191...|
|    1|       1.0|[0.35359258305433...|
|    0|       1.0|[0.35359258305433...|
|    1|       1.0|[0.35359258305433...|
|    1|       1.0|[0.35359258305433...|
|    1|       1.0|[0.35359258305433...|
|    1|       1.0|[0.35359258305433...|
|    0|       1.0|[0.35359258305433...|
|    1|       1.0|[0.34909090909090...|
|    1|       1.0|[0.35359258305433...|
|    1|       1.0|[0.34909090909090...|
|    1|       1.0|[0.35359258305433...|
|    1|       1.0|[0.35359258305433...|
|    1|       1.0|[0.34909090909090...|
+-----+----------+--------------------+
only showing top 20 rows



In [147]:
# create confusion matrix 
pred.groupBy('label','prediction').count().show()

# Calculate the individual elements of the confusion matrix
TN = pred.filter('label = 0 AND prediction = 0').count()
TP = pred.filter('label = 1 AND prediction = 1').count()
FN = pred.filter('label = 1 AND prediction = 0').count()
FP = pred.filter('label = 0 AND prediction = 1').count()

accuracy = (TN+TP)/(TN+TP+FN+FP)
accuracy

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1299|
|    0|       0.0| 2513|
|    1|       1.0| 3376|
|    0|       1.0| 2097|
+-----+----------+-----+



0.634248788368336

In [150]:
from pyspark.ml.classification import LogisticRegression

logistic = LogisticRegression().fit(flights_train)

pred = logistic.transform(flights_test)

# Calculate the individual elements of the confusion matrix
TN = pred.filter('label = 0 AND prediction = 0').count()
TP = pred.filter('label = 1 AND prediction = 1').count()
FN = pred.filter('label = 1 AND prediction = 0').count()
FP = pred.filter('label = 0 AND prediction = 1').count()

accuracy = (TN+TP)/(TN+TP+FN+FP)
print(accuracy)

pred.groupBy('label','prediction').count().show()

0.6063543349488422
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1621|
|    0|       0.0| 2576|
|    1|       1.0| 3054|
|    0|       1.0| 2034|
+-----+----------+-----+



In [151]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

# Calculate precision and recall

precision = TP/(TP+FP)
recall = TP /(TP+FN)

print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision


multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(pred,{multi_evaluator.metricName:"weightedPrecision"})

binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(pred,{binary_evaluator.metricName:'areaUnderROC'})

precision = 0.60
recall    = 0.65
